# Use k-means clustering to find neighborhoods

This notebook should be run after cell phenotype labels have been generated. This notebook walks through generation of neigborhoods using k-means clustering.

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
from scipy.stats import zscore

from ark.utils import spatial_analysis_utils, io_utils, data_utils, plot_utils, load_utils
from ark.analysis import spatial_analysis, visualize

import ark.settings as settings

## 1. Pre-process data

Check `settings.py` and confirm that the correct column names are specified. Importantly, the cell phenotype labels generated from cell clustering should be specified by the "CELL_TYPE" variable in `settings.py`. The k-means cluster IDs will be written to the "KMEANS_CLUSTER" column specified in `settings.py`.

* `base_dir`: the path to all of your imaging data. Should contain a directory for your images, segmentations, and cell table (generated from `1_Segment_Image_Data.ipynb`). This directory will also store all of the directories/files created during neighborhood analysis.
* `segmentation_dir`: the path to the directory containing your segmentations (generated from `1_Segment_Image_Data.ipynb`).
* `cell_table_path`: the path to the cell table that contains columns for fov, cell label, and cell phenotype (generated from `3_Cluster_Cells.ipynb`)
* `spatial_analysis_dir`: the path to the output directory to store all files created during neighborhood analysis

### 1.1 Read data

In [ ]:
base_dir = "../data/example_dataset/"
seg_output = os.path.join(base_dir, "deepcell_output")
cell_table_path = os.path.join(base_dir, "single_cell_output", "cell_table_size_normalized_cell_labels.csv")
spatial_analysis_dir = os.path.join(base_dir, "kmeans_neighborhood_analysis")

# Create output directory
if not os.path.exists(spatial_analysis_dir):
    os.makedirs(spatial_analysis_dir)

In [ ]:
# Validate file paths
io_utils.validate_paths([
    base_dir,
    spatial_analysis_dir,
])

In [ ]:
# Read cell table, only fovs in the cell table will be included in the analysis
all_data = pd.read_csv(cell_table_path)
all_fovs = list(all_data[settings.FOV_ID].drop_duplicates().values)

# Load label maps from segmentation from which the distance matrix will be computed, change suffix if not "_feature_0.tif"
label_maps = load_utils.load_imgs_from_dir(data_dir=seg_output, files=[x+"_feature_0.tif" for x in all_fovs],
                                           xr_channel_names=['segmentation_label'],
                                           trim_suffix='_feature_0')

# Create dictionary object with the respective distance matrices for the fovs
dist_mats = spatial_analysis_utils.calc_dist_matrix(label_maps)

### 1.2 Find neighbors for each cell

For each cell in the cell table, count the number of each cell phenotype within a certain distance of that cell. If a cell has no other cells within the distance specfied by `dist_lim`, that cell is excluded from the analysis.

In [ ]:
# Determine number of each cell type in specified distance limit around each cell
neighbor_counts, neighbor_freqs = spatial_analysis.create_neighborhood_matrix(all_data, dist_mats, distlim=50)

# Save neighbor matrices
neighbor_counts.to_csv(os.path.join(spatial_analysis_dir, "neighborhood_counts.csv"), index=False)
neighbor_freqs.to_csv(os.path.join(spatial_analysis_dir, "neighborhood_freqs.csv"), index=False)

Choose whether to find neighborhoods using count (number of each cell phenotype) or frequency (number of each cell phenotype divided by total number of close neighbors for each cell).

In [ ]:
# This variable can be "counts" or "frequency"
neighborhood_method = "counts"

if neighborhood_method == "counts":
    input_features = neighbor_counts
elif neighborhood_method == "frequency":
    input_features = neighbor_freqs
else:
    print("neighborhood_method must be either 'counts' or 'frequency'")

## 2. Find optimal k using various metrics

Use various metrics to determine the optimal k for k-means clustering. k's ranging from 2 to `max_k` will be tested.

In [ ]:
# The largest k that will be tested
max_k = 10

### 2.1 Inertia

Inertia is calculated by measuring the distance between each data point and its centroid, squaring this distance, and summing these squares across one cluster.

In [ ]:
neighbor_inertia = spatial_analysis.compute_cluster_metrics_inertia(input_features, max_k=max_k)

# Use the elbow curve method to choose the optimal k
visualize.visualize_neighbor_cluster_metrics(neighbor_inertia, metric_name="inertia")

### 2.2 Silhouette score

The Silhouette score is calculated using the mean intra-cluster distance (a) and the mean nearest-cluster distance (b) for each sample. The Silhouette score for a sample is `(b - a) / max(a, b)`.

Silhouette score can be an expensive calculation, so depending on the size of your dataset, you may want to subsample the number of cells in each cluster for this calculation. Each neighborhoood cluster will be sampled down to the number of cells specified by `num_cells_per_cluster`. Set this variable as `None` to use all cells.

In [ ]:
# Subsample each phenotype to avoid long runtimes
num_cells_per_cluster = 1000

neighbor_silhouette_scores = spatial_analysis.compute_cluster_metrics_silhouette(input_features,
                                                                                 max_k=max_k,
                                                                                 subsample=num_cells_per_cluster)

# Use the elbow curve method to choose the optimal k
visualize.visualize_neighbor_cluster_metrics(neighbor_silhouette_scores, metric_name="silhouette")

## 3. Generate cluster results

While the metrics above algorithmically help to choose a k, visualizing the clustering results may help choose the best k. Iterate between steps 3 and 4 to choose the optimal cluster number.

In [ ]:
# Set k value here based on results from graphs above and visualization results
k = 6

In [ ]:
# These channels will be excluded when generating table of mean marker expression per neighborhood cluster
excluded_channels = ["Au","Fe","Noodle","dsDNA"]

# k-means clustering
all_data_cluster_labeled, num_cell_type_per_cluster, mean_marker_exp_per_cluster = \
    spatial_analysis.generate_cluster_matrix_results(
        all_data, input_features, cluster_num=k, excluded_channels=excluded_channels)

## 4. Visualize cluster results

### 4.1 Cluster composition (z-scored for each cell type)

In [ ]:
# Combine composition data with neighborhood labels
merged_dat = input_features.merge(all_data_cluster_labeled[[settings.FOV_ID, settings.CELL_LABEL, settings.KMEANS_CLUSTER]],
                                  on=[settings.FOV_ID, settings.CELL_LABEL])
# Get mean composition of cells in each neighborhood
merged_dat = merged_dat.drop([settings.FOV_ID, settings.CELL_LABEL], axis=1)
mean_dat = merged_dat.groupby(settings.KMEANS_CLUSTER, as_index=False).mean()

# Draw heatmap
mean_dat_values = mean_dat.drop(settings.KMEANS_CLUSTER, axis=1)
visualize.draw_heatmap(data=mean_dat_values.apply(zscore).values,
                       x_labels=["Cluster"+str(x) for x in mean_dat[settings.KMEANS_CLUSTER].values],
                       y_labels=mean_dat.drop(settings.KMEANS_CLUSTER, axis=1).columns.values,
                       center_val=0)

### 4.2 Number of each cell type per neighborhood (z-scored for each cell type)

In [ ]:
visualize.draw_heatmap(data=num_cell_type_per_cluster.apply(zscore).values,
                       x_labels=num_cell_type_per_cluster.index.values,
                       y_labels=num_cell_type_per_cluster.columns.values,
                       center_val=0)

### 4.3 Mean marker expression per neighborhood (z-scored for each marker)

In [ ]:
visualize.draw_heatmap(data=mean_marker_exp_per_cluster.apply(zscore).values,
                       x_labels=mean_marker_exp_per_cluster.index.values,
                       y_labels=mean_marker_exp_per_cluster.columns.values,
                       center_val=0)

### 4.4 Overlay segmentation with neighborhood clusters

It may be helpful to view a few example overlays to assess clustering. For example, for a dataset of lymph nodes, we would expect to see neighborhoods corresponding to the T cell zone, B cell zone, germinal centers, etc.

In [ ]:
# select fovs to display 
fovs = ['2022-03-04T14-42-19-fov-1-scan-1', '2022-03-08T08-10-02-fov-2-scan-1'] 
cluster_labels_xr = data_utils.label_cells_by_cluster(fovs, all_data_cluster_labeled, label_maps) 
plot_utils.plot_neighborhood_cluster_result(cluster_labels_xr, fovs, k=k, save_dir=None)

## 5. Save data

### 5.1 Save data

After running k-means using the optimal k, re-save the cell table with neighborhood cluster ID appended. If a cell has no other cells within the distance specfied by `dist_lim`, that cell is excluded from this table.

In [ ]:
#  Save cell table
new_table_path = cell_table_path.replace(".csv","_kmeans_nh.csv")
all_data_cluster_labeled.to_csv(new_table_path, index=False)

# Save number of each cell type and mean marker expression per neighborhood
num_cell_type_per_cluster.to_csv(os.path.join(spatial_analysis_dir, "neighborhood_cell_type.csv"), index=True, index_label=settings.KMEANS_CLUSTER)
mean_marker_exp_per_cluster.to_csv(os.path.join(spatial_analysis_dir, "neighborhood_marker.csv"), index=True, index_label=settings.KMEANS_CLUSTER)

### 5.2 Save overay for all FOVs

In [ ]:
# Directory to save overlays
overlay_out_dir = os.path.join(spatial_analysis_dir, "overlays")
if not os.path.exists(overlay_out_dir):
    os.makedirs(overlay_out_dir)

# Create and save overlays
cluster_labels_xr = data_utils.label_cells_by_cluster(all_fovs, all_data_cluster_labeled, label_maps) 
data_utils.save_fov_images(
    all_fovs,
    overlay_out_dir,
    cluster_labels_xr,
    name_suffix='_kmeans_nh'
)

### 5.3 Save images for Mantis

Mantis Viewer is a visualization tool for multi-dimensional imaging in pathology. Learn more about Mantis Viewer in the [README](../README.md#mantis-viewer).

In [ ]:
# Make mapping table for mantis
mapping_tab = pd.DataFrame({'metacluster':list(range(1,k+1)), 'mc_name':['Cluster'+str(x) for x in list(range(1,k+1))]})

# Change path names as necessary
plot_utils.create_mantis_project(
    fovs=all_fovs,
    mantis_project_path=os.path.join(spatial_analysis_dir, "mantis"),
    img_data_path=os.path.join(base_dir, "input_data", "single_channel_inputs"),
    mask_output_dir=overlay_out_dir,
    mapping=mapping_tab,
    seg_dir=seg_output,
    mask_suffix="_kmeans_nh",
    img_sub_folder=""
)